%md
# 1. Pre-procesamiento

In [ ]:
%spark.pyspark
# Leer el archivo CSV en un DataFrame de Spark
df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("sep", "|") \
    .option("encoding", "ISO-8859-1") \
    .option("inferSchema", "true") \
    .load("/notebook/data/Det_declara_2022.csv")
df.count()

#CAMBIAR LA RUTA DE LECTURA POR PARQUECT

#df = spark.read("hdfs:PATH_ARCHIVO_PARKET")

In [ ]:
%spark.pyspark
#from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, split, array_intersect, lpad
from pyspark.sql.types import StringType, ArrayType
import re
#from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
%spark.pyspark
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def preprocesar_texto(texto):
    """
    Preprocesa el texto:
    - Convierte a minúsculas
    - Elimina acentos
    - Elimina caracteres especiales
    - Elimina stopwords en español
    """
    if texto is None:
        return ""
    
    texto = str(texto).lower()
    #texto = unidecode(texto)
    texto = re.sub(r'[^a-zA-Z\s]', ' ', texto)
    stop_words = set(stopwords.words('spanish'))
    palabras = texto.split()
    palabras = [palabra for palabra in palabras if palabra not in stop_words and len(palabra) > 2]
    return palabras
    #return ' '.join(palabras)  # Retorna una lista de palabras clave

# Registrar UDF en Spark
preprocesar_udf = udf(preprocesar_texto, ArrayType(StringType()))

In [ ]:
%spark.pyspark
text_result = preprocesar_texto("PESTACION DE SOLDADURA,S/M,T12-XS,")
#set(text_result.split())
text_result

In [ ]:
%spark.pyspark
def filtrar_por_keywords_optimizado(df1, df2):
    """
    Filtra DataFrames por coincidencia de palabras clave usando PySpark.
    """
    # Merge de los DataFrames en base a NUM_PARTNANDI
    df_merged = df1.join(df2, df1.NUM_PART_NANDI == df2.NUM_PARTNANDI, "inner")
    
    # Aplicar preprocesamiento a las columnas de descripción
    df_merged = df_merged.withColumn("keywords_1", preprocesar_udf(col("DES_NOM_COMERC")))
    df_merged = df_merged.withColumn("keywords_2", preprocesar_udf(col("DES_COMER")))
    
    # Filtrar por intersección de palabras clave
    df_merged = df_merged.withColumn("tiene_keyword_comun", array_intersect(col("keywords_1"), col("keywords_2")))
    df_merged = df_merged.filter(col("tiene_keyword_comun").isNotNull())
    
    # Seleccionar columnas finales
    resultado = df_merged.drop("keywords_1", "keywords_2", "tiene_keyword_comun")
    
    return resultado

In [ ]:
%spark.pyspark
df.show(5)

In [ ]:
%spark.pyspark
df_dams = df.withColumn("NUM_PARTNANDI", lpad(col("NUM_PARTNANDI"), 10, "0"))
df_dams.count()

In [ ]:
%spark.pyspark
df_dams_verde = df_dams.filter(df_dams['COD_CANAL'] == 'V')
df_dams_verde.count()

In [ ]:
%spark.pyspark
df_dams_doc = df_dams.filter(df_dams['COD_CANAL'] == 'D')
df_dams_doc.count()

In [ ]:
%spark.pyspark
df_dams_fisi = df_dams.filter(df_dams['COD_CANAL'] == 'F')
df_dams_fisi.count()

In [ ]:
%spark.pyspark

df_class_index = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/notebook/data/INDICE_CLASIFICACION.csv")
df_class_index.count()


In [ ]:
%spark.pyspark
df_class_index.printSchema()

In [ ]:
%spark.pyspark
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [ ]:
%spark.pyspark
df_class_index.show(10)

In [ ]:
%spark.pyspark
#df_pruebas = df_class_index.filter((col("index") >= 10) & (col("index") <= 20))
df_pruebas = df_class_index

from pyspark.sql import Window
from pyspark.sql.functions import row_number

# Crear una columna con números de fila
window_spec = Window.orderBy("NUM_PART_NANDI")  # Reemplaza "COLUMNA_DE_ORDEN" con una columna válida
df_with_row_num = df_pruebas.withColumn("row_num", row_number().over(window_spec))


In [ ]:
%spark.pyspark
df_with_row_num.count()

In [ ]:
%spark.pyspark
# Filtrar para obtener las filas 10 a 20
df_slice = df_with_row_num.filter((col("row_num") >= 10) & (col("row_num") <= 4020)).drop("row_num")

# Mostrar el resultado
df_slice.show()

In [ ]:
%spark.pyspark


In [ ]:
%spark.pyspark
df_slice = df_slice.withColumn("PARTNANDI_LEN", 
    length(trim(col("NUM_PART_NANDI")))
)
df_slice.show()

In [ ]:
%spark.pyspark
from pyspark.sql.functions import col, length, trim, row_number
from pyspark.sql.window import Window

# Convert and filter using correct column name "NUM_PART_NANDI"
df_slice = df_slice.filter(
    (col("PARTNANDI_LEN") == 10) & 
    (col("IND_VIGENCIA") == "S")  # Asumiendo que IND_DEL es equivalente a IND_VIGENCIA
)
df_slice.show()

In [ ]:
%spark.pyspark
# Group by using correct column name
window_spec = Window.partitionBy("NUM_PART_NANDI").orderBy("NUM_PART_NANDI")

df_final = df_slice \
    .withColumn("row_number", row_number().over(window_spec))
    
df_final.filter(col("row_number") > 1).show()

In [ ]:
%spark.pyspark
    .filter(col("row_number") == 1) \
    .drop("row_number", "PARTNANDI_LEN")

# Show results
print("\nPrimeras 5 filas del resultado:")
df_final.show(5)

# Count records
print("\nTotal de registros:", df_final.count())

In [ ]:
%spark.pyspark
    .filter(col("row_number") == 1) \
    .drop("row_number", "PARTNANDI_LEN")

# Show results
print("\nPrimeras 5 filas del resultado:")
df_final.show(5)

# Count records
print("\nTotal de registros:", df_final.count())

In [ ]:
%spark.pyspark


In [ ]:
%spark.pyspark


In [ ]:
%spark.pyspark


%md
## FIN PRUEBAS

In [ ]:
%spark.pyspark
from pyspark.sql.functions import col, length, trim, row_number
from pyspark.sql.window import Window

# Convert and filter using correct column name "NUM_PARTNANDI"
df_filtered = df_class_index.withColumn("PARTNANDI_LEN", 
    length(trim(col("NUM_PART_NANDI")))
).filter(
    (col("PARTNANDI_LEN") == 10) & 
    (col("IND_VIGENCIA") == "S")  # Asumiendo que IND_DEL es equivalente a IND_VIGENCIA
)

# Group by using correct column name
window_spec = Window.partitionBy("NUM_PART_NANDI").orderBy("NUM_PART_NANDI")

df_final = df_filtered \
    .withColumn("row_number", row_number().over(window_spec)) \
    .filter(col("row_number") == 1) \
    .drop("row_number", "PARTNANDI_LEN")

# Show results
print("\nPrimeras 5 filas del resultado:")
df_final.show(5)

# Count records
print("\nTotal de registros:", df_final.count())

In [ ]:
%spark.pyspark
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

df_dams_doc_filt = filtrar_por_keywords_optimizado(df_final, df_dams_doc)
# Mostrar algunos resultados para verificar
print("\nPrimeras 5 filas del resultado:")
df_dams_doc_filt.show(5)

# Contar registros para verificar
print("\nTotal de registros:", df_dams_doc_filt.count())

In [ ]:
%spark.pyspark
df_dams_verde_filt = filtrar_por_keywords_optimizado(df_final, df_dams_verde)
# Mostrar algunos resultados para verificar
print("\nPrimeras 5 filas del resultado:")
df_dams_verde_filt.show(5)

# Contar registros para verificar
print("\nTotal de registros:", df_dams_verde_filt.count())

In [ ]:
%spark.pyspark
df_dams_fisi_filt = filtrar_por_keywords_optimizado(df_final, df_dams_fisi)
# Mostrar algunos resultados para verificar
print("\nPrimeras 5 filas del resultado:")
df_dams_fisi_filt.show(5)

# Contar registros para verificar
print("\nTotal de registros:", df_dams_fisi_filt.count())

%md
**Guardar csv del canal documentario**

In [ ]:
%spark.pyspark
# Guardar el DataFrame como CSV
df_dams_doc_filt.write \
    .mode("overwrite") \
    .option("header", "true") \
    .option("encoding", "UTF-8") \
    .csv("/notebook/data/cleaned/dams_cleaned_doc.csv")


%md
# 2. Entrenamiento


In [ ]:
%spark.pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, udf, lit
from pyspark.sql.types import StringType
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [ ]:
%spark.pyspark
df_dams_doc_filt = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/notebook/data/cleaned/dams_cleaned_doc.csv")

In [ ]:
%spark.pyspark
# window_spec = Window.orderBy("NUM_PART_NANDI")
# df_with_row_num = df_dams_doc_filt.withColumn("row_num", row_number().over(window_spec))

# df_dams_doc_filt = df_with_row_num.filter((col("row_num") >= 120000) & (col("row_num") <= 150000)).drop("row_num")
# df_dams_doc_filt.show()

In [ ]:
%spark.pyspark
print(df_dams_doc_filt.count())
print(df_dams_doc_filt.printSchema())

%md
## 2.1 Muestreo de datos

In [ ]:
%spark.pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace, udf, lit
from pyspark.sql.types import StringType
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [ ]:
%spark.pyspark
df_dams_proc = df_dams_doc_filt.select("DES_COMER", "NUM_PARTNANDI")

In [ ]:
%spark.pyspark

df_dams_proc = df_dams_proc.withColumn("NUM_PARTNANDI", F.lpad(col("NUM_PARTNANDI").cast("string"), 10, "0"))
df_dams_proc = df_dams_proc.withColumn("CLASE", col("NUM_PARTNANDI").substr(1, 2))
df_dams_proc.show(5)

In [ ]:
%spark.pyspark
# Crear un mapeo de clases a números
class_mapping = {clase: idx for idx, clase in enumerate(df_dams_proc.select("CLASE").distinct().rdd.flatMap(lambda x: x).collect())}
mapping_expr = F.create_map([F.lit(x) for pair in class_mapping.items() for x in pair])

# Aplicar el mapeo
df_dams_proc = df_dams_proc.withColumn("CLASE_NUM", mapping_expr[col("CLASE")])
df_dams_proc.show(5)

In [ ]:
%spark.pyspark
df_distinct = df_dams_proc.select('CLASE_NUM').distinct()
df_distinct.count()

In [ ]:
%spark.pyspark
print("Distribución de clases en el dataset:")
df_dams_proc.groupBy("CLASE_NUM", "CLASE").count().orderBy("count").show(60)

In [ ]:
%spark.pyspark
df_dams_proc.printSchema()

In [ ]:
%spark.pyspark
capitulos = ["62", "61", "83", "84"]

df_dams_proc_filtrado = df_dams_proc.filter(col("CLASE").isin(capitulos))

In [ ]:
%spark.pyspark
df_dams_proc_filtrado.groupBy("CLASE_NUM", "CLASE").count().orderBy("count").show()

In [ ]:
%spark.pyspark


In [ ]:
%spark.pyspark
from pyspark.sql.functions import col, lower, regexp_replace, udf, lit
from pyspark.sql.types import StringType
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [ ]:
%spark.pyspark
from pyspark.sql import Window
from pyspark.sql.functions import col, lower, regexp_replace, trim, row_number, lit

def preparar_dataset_clasificacion(df, clases_seleccionadas, muestras_por_clase=100):
    """
    Prepara un dataset de clasificación en PySpark.
    
    Parámetros:
    -----------
    df : DataFrame de PySpark
        DataFrame original con columnas [DES_COMER] y CLASE_NUM.
    clases_seleccionadas : list
        Lista de clases a incluir en el dataset.
    muestras_por_clase : int, opcional
        Número de muestras a obtener por cada clase (default: 100).
    
    Retorna:
    --------
    DataFrame de PySpark
        Dataset preparado para entrenamiento de red neuronal.
    """
    # Filtrar solo las clases seleccionadas
    df_filtrado = df.filter(col("CLASE_NUM").isin(clases_seleccionadas))
    
    # Limpiar descripciones
    df_filtrado = df_filtrado.withColumn("DESCR_LIMPIA", trim(lower(regexp_replace(col("DES_COMER"), "\\s+", " "))))
    
    # Eliminar descripciones duplicadas dentro de cada clase
    # Agregar una columna de orden (puedes usar una columna existente o una constante)
    window_spec = Window.partitionBy("CLASE_NUM", "DESCR_LIMPIA").orderBy(lit(1))
    df_unico = df_filtrado.withColumn("row_num", row_number().over(window_spec)) \
        .filter(col("row_num") == 1) \
        .drop("row_num")
    
    # Preparar el dataset final con muestras equitativas
    dataset_final = []
    
    for clase in clases_seleccionadas:
        # Obtener muestras de la clase actual
        muestras_clase = df_unico.filter(col("CLASE_NUM") == clase)
        
        # Si hay menos muestras que las solicitadas, usar todas
        if muestras_clase.count() <= muestras_por_clase:
            dataset_final.append(muestras_clase)
        else:
            # Muestreo aleatorio estratificado
            muestras_seleccionadas = muestras_clase.sample(fraction=1.0, seed=42).limit(muestras_por_clase)
            dataset_final.append(muestras_seleccionadas)
    
    # Concatenar todas las muestras
    dataset_preparado = dataset_final[0]
    for df_clase in dataset_final[1:]:
        dataset_preparado = dataset_preparado.union(df_clase)
    
    # Verificar la distribución de clases
    print("Distribución de clases en el dataset:")
    dataset_preparado.groupBy("CLASE_NUM").count().show()
    
    return dataset_preparado

In [ ]:
%spark.pyspark
#clases_muestreo = [0, 1, 2, 3]
# clases_muestreo = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 
#                     28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
#                     54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79,
#                     80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96]

clases = [16, 39, 41, 31]

dataset_preparado = preparar_dataset_clasificacion(df_dams_proc_filtrado, clases_muestreo, muestras_por_clase=500)


In [ ]:
%spark.pyspark
dataset_preparado.write \
    .mode("overwrite") \
    .option("header", "true") \
    .option("encoding", "UTF-8") \
    .csv("/notebook/data/cleaned/dams_train_test.csv")

%md
# 3. Entrenamiento


In [ ]:
%spark.pyspark
df_dams_doc_filt = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/notebook/data/dams_muestras_doc.csv")

In [ ]:
%spark.pyspark
# Verificar la distribución de clases
print("Distribución de clases en el dataset:")
df_dams_doc_filt.groupBy("CLASE_NUM", "CLASE").count().show()

In [ ]:
%spark.pyspark
df_dams_doc_filt.show(5)

In [ ]:
%spark.pyspark
class_mapping = {61: 0, 84: 1, 62: 2, 85: 3}
class_mapping

%md
# 4. Entrenamiento con PyTorch

In [ ]:
%spark.pyspark
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
%spark.pyspark
class ArancelDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [ ]:
%spark.pyspark
class TariffClassifier(nn.Module):
    def __init__(self, n_classes):
        super(TariffClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.drop1 = nn.Dropout(p=0.3)
        self.drop2 = nn.Dropout(p=0.2)
        self.drop3 = nn.Dropout(p=0.1)
        self.fc1 = nn.Linear(self.bert.config.hidden_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, n_classes)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        x = self.drop1(outputs[1])
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.drop2(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.drop3(x)

        x = self.fc3(x)
        return x

In [ ]:
%spark.pyspark
dataset_preparado = df_dams_doc_filt.toPandas()

In [ ]:
%spark.pyspark
dataset_preparado

In [ ]:
%spark.pyspark
# Split de datos
X_train, X_test, y_train, y_test = train_test_split(
    dataset_preparado['DES_COMER'].values, 
    dataset_preparado['CLASE_NUM'].values,
    test_size=0.2,
    random_state=42
)

In [ ]:
%spark.pyspark
np.array(X_train[:10]), y_train[:10]

In [ ]:
%spark.pyspark
np.array(X_test[:10]), y_test[:10]

In [ ]:
%spark.pyspark
# Inicializar tokenizer y datasets
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
train_dataset = ArancelDataset(X_train, y_train, tokenizer)
test_dataset = ArancelDataset(X_test, y_test, tokenizer)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Configuración del modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TariffClassifier(len(class_mapping))
model = model.to(device)

# Entrenamiento
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()
epochs = 1

In [ ]:
%spark.pyspark
# Función de entrenamiento
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    
    for batch in tqdm(loader):
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

In [ ]:
%spark.pyspark
# Función de evaluación
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    predictions = []
    true_labels = []
    
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            true_labels.extend(labels.cpu().tolist())
    
    return total_loss / len(loader), predictions, true_labels

In [ ]:
%spark.pyspark
train_losses = []
val_losses = []
# Entrenamiento del modelo
print("Iniciando entrenamiento...")
for epoch in range(epochs):
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, predictions, true_labels = evaluate(model, test_loader, criterion, device)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    print(f'Epoch {epoch + 1}/{epochs}')
    print(f'Training Loss: {train_loss:.4f}')
    print(f'Validation Loss: {val_loss:.4f}')

In [ ]:
%spark.pyspark
import matplotlib.pyplot as plt

plt.plot(train_losses, label="Train Loss")
plt.plot(test_losses, label="Test Loss")
plt.xlabel("Épocas")
plt.ylabel("Pérdida")
plt.title("Convergencia del Modelo")
plt.legend()
plt.grid()
plt.show()

In [ ]:
%spark.pyspark
# 4. Evaluación
print("\nEvaluación del modelo:")
reverse_class_mapping = {v: k for k, v in class_mapping.items()}
predictions_mapped = [reverse_class_mapping[p] for p in predictions]
true_labels_mapped = [reverse_class_mapping[t] for t in true_labels]

# Crear matriz de confusión
plt.figure(figsize=(12, 8))
cm = confusion_matrix(true_labels_mapped, predictions_mapped)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.show()

In [ ]:
%spark.pyspark
# Imprimir reporte de clasificación
print("\nReporte de Clasificación:")
print(classification_report(true_labels_mapped, predictions_mapped))

In [ ]:
%spark.pyspark
torch.save(model.state_dict(), "/notebook/data/models/class-model-nn-doc-v1.pth")

In [ ]:
%spark.pyspark
model = torch.load("/notebook/data/models/class-model-nn-doc-v1.pth")

In [ ]:
%spark.pyspark
# 5. Predicciones de ejemplo
def predict_text(text, model, tokenizer, device):
    model.eval()
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        _, preds = torch.max(outputs, dim=1)
    
    return reverse_class_mapping[preds.item()]

# Ejemplo de predicciones
ejemplos = [
    "MÁQUINAS AUTOMÁTICAS PARA TRATAMIENTO O PROCESAMIENTO DE DATOS, DIGITALES",
    "TEJIDOS DE ALGODÓN CON UN CONTENIDO DE ALGODÓN SUPERIOR O IGUAL AL 85% EN PESO",
    "MEDICAMENTOS PARA USO TERAPÉUTICO O PROFILÁCTICO"
]

print("\nEjemplos de predicciones:")
for texto in ejemplos:
    prediccion = predict_text(texto, model, tokenizer, device)
    print(f"\nTexto: {texto}")
    print(f"Capítulo predicho: {prediccion}")

In [ ]:
%python
import json
import sys
import os

def convert_zpln_to_json(input_file, output_file=None):
    """
    Convierte un archivo .zpln de Zeppelin 0.11.2 a un formato compatible con Zeppelin 0.8
    
    Args:
        input_file (str): Ruta al archivo .zpln
        output_file (str, optional): Ruta para el archivo .json de salida
    """
    try:
        # Leer el archivo .zpln
        with open(input_file, 'r', encoding='utf-8') as f:
            notebook = json.load(f)
        
        # Si no se especifica archivo de salida, crear uno basado en el nombre de entrada
        if output_file is None:
            output_file = os.path.splitext(input_file)[0] + '.json'
        
        # Guardar como JSON
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(notebook, f, indent=2)
            
        print(f"Conversión exitosa. Archivo guardado como: {output_file}")
        
    except json.JSONDecodeError:
        print("Error: El archivo no es un JSON válido")
    except Exception as e:
        print(f"Error durante la conversión: {str(e)}")

In [ ]:
%python
convert_zpln_to_json("/notebook/INCA/NLP/training-torch_2KKB8B1SJ.zpln", "/notebook/INCA/NLP/training-torch-nn-v2.json")

In [ ]:
%python


In [ ]:
%python
